In [52]:
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

Explore the csv, featuring aid received, literacy rates, and GDP per capita. Clean the data. Many columns have ".." signalling that the data doesn't exist. This is particularly true of the literacy rates indicator, which, for many countries, does not record a statistic per year. For now, we'll replace those with zeroes.

In [64]:
country_aid_gdp_lit_df = pd.read_csv("data/2nd_try_aid_gdp_lit.csv")

In [65]:
#Reshape table so each year for each country is its own row. 
df1 = country_aid_gdp_lit_df.set_index(['Country Name', 'Series Name'])
df1.columns.name = 'year'
df1 = df1.stack().unstack('Series Name')

In [66]:
df1.head(3)

Series Name       GDP per capita (current US$)  \
Country Name year                                
Afghanistan  1990                           ..   
             1991                           ..   
             1992                           ..   

Series Name       Literacy rate, adult total (% of people ages 15 and above)  \
Country Name year                                                              
Afghanistan  1990                                                 ..           
             1991                                                 ..           
             1992                                                 ..           

Series Name       Net official development assistance and official aid received (current US$)  
Country Name year                                                                              
Afghanistan  1990                                          121720000                           
             1991                                          507530000                           
             1992                                          204900000

In [67]:
#replace ".." with zeros
df1 = df1.replace("..", "0")

In [68]:
df1.head(3)

Series Name       GDP per capita (current US$)  \
Country Name year                                
Afghanistan  1990                            0   
             1991                            0   
             1992                            0   

Series Name       Literacy rate, adult total (% of people ages 15 and above)  \
Country Name year                                                              
Afghanistan  1990                                                  0           
             1991                                                  0           
             1992                                                  0           

Series Name       Net official development assistance and official aid received (current US$)  
Country Name year                                                                              
Afghanistan  1990                                          121720000                           
             1991                                          507530000                           
             1992                                          204900000

In [69]:
#lets take a look at the dtypes
df1.dtypes

Series Name
GDP per capita (current US$)                                                   object
Literacy rate, adult total (% of people ages 15 and above)                     object
Net official development assistance and official aid received (current US$)    object
dtype: object

In [70]:
#all objects? Need to change that, also need to change column name
df1 = df1.reset_index()
df1.dtypes

Series Name
Country Name                                                                   object
year                                                                           object
GDP per capita (current US$)                                                   object
Literacy rate, adult total (% of people ages 15 and above)                     object
Net official development assistance and official aid received (current US$)    object
dtype: object

In [71]:
df1.index.names = ['id']
df1.columns = ['country', 'year', 'gdp_per_capita', 'literacy_rate', 'net_aid_received']

In [85]:
#time to change the column types
df1['country'] = df1['country'].astype('str')
df1['year'] = df1['year'].astype("int32")
df1['gdp_per_capita'] = df1['gdp_per_capita'].astype('float64')
df1['literacy_rate'] = df1['literacy_rate'].astype('float64')
df1['net_aid_received'] = df1['net_aid_received'].astype('float64')

In [86]:
df1.head(3)

,country,year,gdp_per_capita,literacy_rate,net_aid_received
id,,,,,
0,Afghanistan,1990,0.0,0.0,121720000.0
1,Afghanistan,1991,0.0,0.0,507530000.0
2,Afghanistan,1992,0.0,0.0,204900000.0


In [87]:
df1.dtypes

country              object
year                  int32
gdp_per_capita      float64
literacy_rate       float64
net_aid_received    float64
dtype: object

In [88]:
df1.to_csv("clean_data/gdp_lit_aid.csv")

Now it's time to investigate aid spending per sector by year csv. Should not be as big as the previous csv. Move from wide to long. 

In [338]:
df2 = pd.read_csv("data/aid_spending_by_sector_per_yer.csv")

In [339]:
df2.head(5)

,Sector(s),Time Period,1990,1991,1992,1993,1994,1995,1996,1997,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,"I.1. Education, Total",NaN,5664.71,4973.52,4793.35,4672.43,5376.85,6054.95,5261.22,4683.23,...,8440.51,9417.38,9329.86,9318.94,9780.09,8793.18,8227.74,9157.74,8050.54,8635.15
1,"I.2. Health, Total",NaN,1613.27,1424.84,1617.43,1654.98,1831.49,2171.42,2283.51,1667.18,...,4447.27,5077.02,4990.22,5108.05,6012.12,6160.02,5938.76,5324.30,6094.47,6393.36
2,"I.5. Government & Civil Society, Total",NaN,1735.06,1387.19,1717.47,1300.48,1425.16,1792.34,1496.31,1130.09,...,11678.39,13221.55,13325.07,13983.96,14531.76,13014.89,12526.83,12952.03,11936.95,11924.34
3,"II. Economic Infrastructure & Services, Total",NaN,8382.07,12182.65,8038.75,9591.39,10805.58,12880.17,11512.02,10127.12,...,12242.26,19522.05,15348.72,19487.56,16998.42,17974.14,20836.01,22160.40,22234.71,22004.57
4,"II.5. Business & Other Services, Total",NaN,394.33,308.94,430.42,346.42,1206.94,659.63,642.43,1008.88,...,1443.38,1631.89,1106.67,1309.79,1746.30,941.00,1245.62,2152.68,1238.58,1385.94


In [340]:
#lets drop the time period column and reshape the whole thing with years as rows
df2 = df2.drop(['Time Period'], axis=1)

In [341]:
df2 = df2.set_index(['Sector(s)'])

In [342]:
df2.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Sector(s),,,,,,,,,,,,,,,,,,,,,
"I.1. Education, Total",5664.71,4973.52,4793.35,4672.43,5376.85,6054.95,5261.22,4683.23,4492.27,5038.70,...,8440.51,9417.38,9329.86,9318.94,9780.09,8793.18,8227.74,9157.74,8050.54,8635.15
"I.2. Health, Total",1613.27,1424.84,1617.43,1654.98,1831.49,2171.42,2283.51,1667.18,1593.50,1973.92,...,4447.27,5077.02,4990.22,5108.05,6012.12,6160.02,5938.76,5324.30,6094.47,6393.36
"I.5. Government & Civil Society, Total",1735.06,1387.19,1717.47,1300.48,1425.16,1792.34,1496.31,1130.09,1946.19,1997.61,...,11678.39,13221.55,13325.07,13983.96,14531.76,13014.89,12526.83,12952.03,11936.95,11924.34
"II. Economic Infrastructure & Services, Total",8382.07,12182.65,8038.75,9591.39,10805.58,12880.17,11512.02,10127.12,7803.86,8062.73,...,12242.26,19522.05,15348.72,19487.56,16998.42,17974.14,20836.01,22160.40,22234.71,22004.57
"II.5. Business & Other Services, Total",394.33,308.94,430.42,346.42,1206.94,659.63,642.43,1008.88,534.92,1444.93,...,1443.38,1631.89,1106.67,1309.79,1746.30,941.00,1245.62,2152.68,1238.58,1385.94


In [343]:
df2 = df2.T

In [344]:
df2.index.name = 'year'

In [345]:
df2 = df2.reset_index()

In [346]:
df2.head()

Sector(s),year,"I.1. Education, Total","I.2. Health, Total","I.5. Government & Civil Society, Total","II. Economic Infrastructure & Services, Total","II.5. Business & Other Services, Total","III. Production Sectors, Total","IV. Multi-Sector / Cross-Cutting, Total"
0,1990,5664.71,1613.27,1735.06,8382.07,394.33,6878.85,1871.51
1,1991,4973.52,1424.84,1387.19,12182.65,308.94,6199.57,1524.03
2,1992,4793.35,1617.43,1717.47,8038.75,430.42,8069.91,2041.24
3,1993,4672.43,1654.98,1300.48,9591.39,346.42,5780.78,2045.15
4,1994,5376.85,1831.49,1425.16,10805.58,1206.94,5265.10,1965.18


In [347]:
df2.dtypes

Sector(s)
year                                              object
I.1. Education, Total                            float64
I.2. Health, Total                               float64
I.5. Government & Civil Society, Total           float64
II. Economic Infrastructure & Services, Total    float64
II.5. Business & Other Services, Total           float64
III. Production Sectors, Total                   float64
IV. Multi-Sector / Cross-Cutting, Total          float64
dtype: object

In [348]:
df2.index.names = ["id"]
df2.head(3)

Sector(s),year,"I.1. Education, Total","I.2. Health, Total","I.5. Government & Civil Society, Total","II. Economic Infrastructure & Services, Total","II.5. Business & Other Services, Total","III. Production Sectors, Total","IV. Multi-Sector / Cross-Cutting, Total"
id,,,,,,,,
0,1990,5664.71,1613.27,1735.06,8382.07,394.33,6878.85,1871.51
1,1991,4973.52,1424.84,1387.19,12182.65,308.94,6199.57,1524.03
2,1992,4793.35,1617.43,1717.47,8038.75,430.42,8069.91,2041.24


In [349]:
df2.dtypes

Sector(s)
year                                              object
I.1. Education, Total                            float64
I.2. Health, Total                               float64
I.5. Government & Civil Society, Total           float64
II. Economic Infrastructure & Services, Total    float64
II.5. Business & Other Services, Total           float64
III. Production Sectors, Total                   float64
IV. Multi-Sector / Cross-Cutting, Total          float64
dtype: object

In [350]:
df2.shape

(27, 8)

In [351]:
df2.columns = ['year', 'education', 'health', 'government', 'economic', 'business', 'production', 'multi-sector']

In [352]:
df2['year'] = df2['year'].astype('int32')

In [353]:
df2.dtypes

year              int32
education       float64
health          float64
government      float64
economic        float64
business        float64
production      float64
multi-sector    float64
dtype: object

In [354]:
df2.head(3)

,year,education,health,government,economic,business,production,multi-sector
id,,,,,,,,
0,1990,5664.71,1613.27,1735.06,8382.07,394.33,6878.85,1871.51
1,1991,4973.52,1424.84,1387.19,12182.65,308.94,6199.57,1524.03
2,1992,4793.35,1617.43,1717.47,8038.75,430.42,8069.91,2041.24


In [355]:
df2 = df2.set_index(['year'])
df2.head()


,education,health,government,economic,business,production,multi-sector
year,,,,,,,
1990,5664.71,1613.27,1735.06,8382.07,394.33,6878.85,1871.51
1991,4973.52,1424.84,1387.19,12182.65,308.94,6199.57,1524.03
1992,4793.35,1617.43,1717.47,8038.75,430.42,8069.91,2041.24
1993,4672.43,1654.98,1300.48,9591.39,346.42,5780.78,2045.15
1994,5376.85,1831.49,1425.16,10805.58,1206.94,5265.10,1965.18


In [356]:
df2.to_csv("clean_data/sector.csv")

Finally, clean and inspect the spending vs life expectancy time csv. The csv uses aggregate indicators. 

In [357]:
df3 = pd.read_csv("data/life_expect.csv")

In [358]:
df3.head(3)

,Country Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],...,2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,IDA only,53.039285,53.301928,53.582540,53.885894,54.209501,54.558191,54.930012,55.314159,55.710239,...,59.858147,60.459556,61.044044,61.599662,62.116124,62.594243,63.040820,63.459617,63.853122,64.222994
1,Lower middle income,59.513275,59.829442,60.183902,60.514125,60.838366,61.157632,61.483625,61.817486,62.150271,...,64.799075,65.165819,65.529030,65.901517,66.262475,66.596613,66.904789,67.193320,67.458766,67.705178
2,Low & middle income,63.106052,63.326673,63.493558,63.653089,63.868162,64.114060,64.404773,64.718892,65.011587,...,67.538034,67.885040,68.212894,68.555213,68.865134,69.178624,69.459854,69.729661,69.970356,70.193261


In [359]:
df3.columns = ['aggregator','1990', '1991','1992', '1993', '1994', '1995', '1996', '1997', '1998','1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

In [360]:
df3.head(3)


,aggregator,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,IDA only,53.039285,53.301928,53.582540,53.885894,54.209501,54.558191,54.930012,55.314159,55.710239,...,59.858147,60.459556,61.044044,61.599662,62.116124,62.594243,63.040820,63.459617,63.853122,64.222994
1,Lower middle income,59.513275,59.829442,60.183902,60.514125,60.838366,61.157632,61.483625,61.817486,62.150271,...,64.799075,65.165819,65.529030,65.901517,66.262475,66.596613,66.904789,67.193320,67.458766,67.705178
2,Low & middle income,63.106052,63.326673,63.493558,63.653089,63.868162,64.114060,64.404773,64.718892,65.011587,...,67.538034,67.885040,68.212894,68.555213,68.865134,69.178624,69.459854,69.729661,69.970356,70.193261


In [361]:
df3 = df3.set_index('aggregator')

In [362]:
df3.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
aggregator,,,,,,,,,,,,,,,,,,,,,
IDA only,53.039285,53.301928,53.582540,53.885894,54.209501,54.558191,54.930012,55.314159,55.710239,56.128856,...,59.858147,60.459556,61.044044,61.599662,62.116124,62.594243,63.040820,63.459617,63.853122,64.222994
Lower middle income,59.513275,59.829442,60.183902,60.514125,60.838366,61.157632,61.483625,61.817486,62.150271,62.469101,...,64.799075,65.165819,65.529030,65.901517,66.262475,66.596613,66.904789,67.193320,67.458766,67.705178
Low & middle income,63.106052,63.326673,63.493558,63.653089,63.868162,64.114060,64.404773,64.718892,65.011587,65.281524,...,67.538034,67.885040,68.212894,68.555213,68.865134,69.178624,69.459854,69.729661,69.970356,70.193261


In [381]:
df2_temp = pd.read_csv('data/aid_spending_by_sector_per_yer.csv')
df2_temp = df2_temp.set_index(['Sector(s)'])

In [386]:
df2_temp = df2_temp.drop(['Time Period'], axis=1)

In [387]:
dd = df2_temp.loc["I.2. Health, Total", :'2015'] 

In [389]:
df3 = df3.append(dd)

In [391]:
df3 = df3.T

In [395]:
df3.index.name = 'year'
df3.columns = ['ida_only', 'lower_middle_income', 'low_middle_income', 'health']

In [396]:
df3.head()

,ida_only,lower_middle_income,low_middle_income,health
year,,,,
1990,53.039285,59.513275,63.106052,1613.27
1991,53.301928,59.829442,63.326673,1424.84
1992,53.582540,60.183902,63.493558,1617.43
1993,53.885894,60.514125,63.653089,1654.98
1994,54.209501,60.838366,63.868162,1831.49


In [398]:
df3.to_csv("clean_data/life_clean.csv")

In [397]:
df3.dtypes

ida_only               float64
lower_middle_income    float64
low_middle_income      float64
health                 float64
dtype: object